In [9]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt # for data visualization
import seaborn as sns # for statistical data visualization
from datetime import datetime

In [10]:
#pip install datasets

In [11]:
import numpy as np
import torch
import torch.utils.data
import scipy
import torch.nn.functional as F
#import datasets
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
#datasets.logging.set_verbosity_error()

In [12]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

In [13]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.optimizers import Adam

In [14]:
import tensorflow
tensorflow.random.set_seed(0)
np.random.seed(0)

In [15]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.layers import Dropout


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
df = pd.read_csv('drive/My Drive/thesis/dataset/final_dataset_with_all_stock.csv [Final one]')

In [18]:
df = df.drop(df.columns[0],axis=1)

In [19]:
#df

In [20]:
df = df.sort_values(by=['stock_symbol', 'Date'])

In [21]:
df

,stock_symbol,Date,Open,Close,High,Low,Volume,macd,tweet_sentiment_count_0,tweet_sentiment_count_1,...,Tuesday,Wednesday,tweet_0_pct,tweet_1_pct,tweet_2_pct,stocktwit_0_pct,stocktwit_1_pct,news_0_pct,news_1_pct,news_2_pct
0,AAPL,2022-03-01,164.699997,163.199997,166.600006,161.970001,83474400,-0.760515,164,125,...,1,0,0.340956,0.259875,0.399168,0.382781,0.617219,0.666667,0.333333,0.000000
1,AAPL,2022-03-02,164.389999,166.559998,167.360001,162.949997,79724800,-0.470632,69,200,...,0,1,0.161972,0.469484,0.368545,0.265512,0.734488,1.000000,0.000000,0.000000
2,AAPL,2022-03-03,168.470001,166.229996,168.910004,165.550003,76678400,-0.274369,103,160,...,0,0,0.213251,0.331263,0.455487,0.339372,0.660628,0.000000,1.000000,0.000000
3,AAPL,2022-03-08,158.820007,157.440002,162.880005,155.800003,131148300,-0.797608,158,503,...,1,0,0.150907,0.480420,0.368672,0.362690,0.637310,0.021277,0.808511,0.170213
4,AAPL,2022-03-09,161.479996,162.949997,163.410004,159.410004,91454900,-0.532939,84,224,...,0,1,0.156716,0.417910,0.425373,0.259504,0.740496,0.000000,0.777778,0.222222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794,WMT,2022-12-21,144.320007,145.179993,145.940002,144.259995,4575500,-1.058663,14,23,...,0,1,0.186667,0.306667,0.506667,0.170732,0.829268,0.166667,0.666667,0.166667
3795,WMT,2022-12-22,144.770004,143.479996,144.979996,141.929993,4360800,-0.992514,9,33,...,0,0,0.105882,0.388235,0.505882,0.258065,0.741935,0.055556,0.777778,0.166667
3796,WMT,2022-12-27,144.039993,143.809998,144.449997,143.190002,3348200,-0.770568,19,25,...,1,0,0.250000,0.328947,0.421053,0.437500,0.562500,0.166667,0.416667,0.416667
3797,WMT,2022-12-28,143.470001,141.289993,143.669998,140.910004,5082100,-0.819899,16,24,...,0,1,0.200000,0.300000,0.500000,0.366667,0.633333,0.111111,0.555556,0.333333


In [22]:
# Close - Open change

def calculate_price_change(group):
    Price_Change = group['Close'] - group['Open']
    group['price_change'] = Price_Change
    return group

group_df = df.groupby('stock_symbol', group_keys=False).apply(calculate_price_change)


In [23]:
def calculate_direction(group):
    Direction = group['price_change'].apply(lambda x: 1 if isinstance(x, (float, int)) and x > 0 else 0 if isinstance(x, (float, int)) and x <= 0 else None)
    group['direction'] = Direction
    return group

group_df = group_df.groupby('stock_symbol', group_keys=False).apply(calculate_direction)


In [24]:
group_df

,stock_symbol,Date,Open,Close,High,Low,Volume,macd,tweet_sentiment_count_0,tweet_sentiment_count_1,...,tweet_0_pct,tweet_1_pct,tweet_2_pct,stocktwit_0_pct,stocktwit_1_pct,news_0_pct,news_1_pct,news_2_pct,price_change,direction
0,AAPL,2022-03-01,164.699997,163.199997,166.600006,161.970001,83474400,-0.760515,164,125,...,0.340956,0.259875,0.399168,0.382781,0.617219,0.666667,0.333333,0.000000,-1.500000,0
1,AAPL,2022-03-02,164.389999,166.559998,167.360001,162.949997,79724800,-0.470632,69,200,...,0.161972,0.469484,0.368545,0.265512,0.734488,1.000000,0.000000,0.000000,2.169998,1
2,AAPL,2022-03-03,168.470001,166.229996,168.910004,165.550003,76678400,-0.274369,103,160,...,0.213251,0.331263,0.455487,0.339372,0.660628,0.000000,1.000000,0.000000,-2.240006,0
3,AAPL,2022-03-08,158.820007,157.440002,162.880005,155.800003,131148300,-0.797608,158,503,...,0.150907,0.480420,0.368672,0.362690,0.637310,0.021277,0.808511,0.170213,-1.380005,0
4,AAPL,2022-03-09,161.479996,162.949997,163.410004,159.410004,91454900,-0.532939,84,224,...,0.156716,0.417910,0.425373,0.259504,0.740496,0.000000,0.777778,0.222222,1.470001,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3794,WMT,2022-12-21,144.320007,145.179993,145.940002,144.259995,4575500,-1.058663,14,23,...,0.186667,0.306667,0.506667,0.170732,0.829268,0.166667,0.666667,0.166667,0.859985,1
3795,WMT,2022-12-22,144.770004,143.479996,144.979996,141.929993,4360800,-0.992514,9,33,...,0.105882,0.388235,0.505882,0.258065,0.741935,0.055556,0.777778,0.166667,-1.290009,0
3796,WMT,2022-12-27,144.039993,143.809998,144.449997,143.190002,3348200,-0.770568,19,25,...,0.250000,0.328947,0.421053,0.437500,0.562500,0.166667,0.416667,0.416667,-0.229996,0
3797,WMT,2022-12-28,143.470001,141.289993,143.669998,140.910004,5082100,-0.819899,16,24,...,0.200000,0.300000,0.500000,0.366667,0.633333,0.111111,0.555556,0.333333,-2.180008,0


In [25]:
group_df.isnull().sum()

stock_symbol                   0
Date                           0
Open                           0
Close                          0
High                           0
Low                            0
Volume                         0
macd                           0
tweet_sentiment_count_0        0
tweet_sentiment_count_1        0
tweet_sentiment_count_2        0
tweet_sentiment_mean           0
stocktwit_sentiment_count_0    0
stocktwit_sentiment_count_1    0
stocktwit_sentiment_mean       0
news_sentiment_count_0         0
news_sentiment_count_1         0
news_sentiment_count_2         0
news_sentiment_mean            0
trends                         0
Null_value_news                0
Null_value_stocktwit           0
week_day                       0
Thursday                       0
Tuesday                        0
Wednesday                      0
tweet_0_pct                    0
tweet_1_pct                    0
tweet_2_pct                    0
stocktwit_0_pct                0
stocktwit_

In [26]:
group_df = group_df.dropna()

In [27]:
group_df.shape

(3799, 36)

In [28]:
group_df.columns

Index(['stock_symbol', 'Date', 'Open', 'Close', 'High', 'Low', 'Volume',
       'macd', 'tweet_sentiment_count_0', 'tweet_sentiment_count_1',
       'tweet_sentiment_count_2', 'tweet_sentiment_mean',
       'stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1',
       'stocktwit_sentiment_mean', 'news_sentiment_count_0',
       'news_sentiment_count_1', 'news_sentiment_count_2',
       'news_sentiment_mean', 'trends', 'Null_value_news',
       'Null_value_stocktwit', 'week_day', 'Thursday', 'Tuesday', 'Wednesday',
       'tweet_0_pct', 'tweet_1_pct', 'tweet_2_pct', 'stocktwit_0_pct',
       'stocktwit_1_pct', 'news_0_pct', 'news_1_pct', 'news_2_pct',
       'price_change', 'direction'],
      dtype='object')

# 1. price, macd, twitter, stocktwit---finished

#2. price, macd, twitter, stocktwit, news



#3. price,macd,twitter,stocktwit,news,trends---

#4. price,macd,twitter,stocktwit,trends---

#5. price,macd,news,trends---

#6. price,macd,twitter---







#7. price,macd,stocktwit---

#8. price,macd,news---

#9. price,macd,trends---

#10. price,macd---

In [29]:
data = group_df[['stock_symbol', 'Date', 'Open', 'Close', 'High', 'Low',
       'macd', 'Thursday', 'Tuesday', 'Wednesday', 'price_change', 
       'direction', 
       'tweet_sentiment_count_0', 'tweet_sentiment_count_1', 
       'tweet_sentiment_count_2', 'tweet_sentiment_mean', 
       'stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1', 
       'stocktwit_sentiment_mean', 
       #'news_sentiment_count_0', 
       #'news_sentiment_count_1', 'news_sentiment_count_2', 
       #'news_sentiment_mean', 
       #'trends', 
       #'Null_value_news',
       'Null_value_stocktwit'
       ]].copy()

In [30]:
data.columns

Index(['stock_symbol', 'Date', 'Open', 'Close', 'High', 'Low', 'macd',
       'Thursday', 'Tuesday', 'Wednesday', 'price_change', 'direction',
       'tweet_sentiment_count_0', 'tweet_sentiment_count_1',
       'tweet_sentiment_count_2', 'tweet_sentiment_mean',
       'stocktwit_sentiment_count_0', 'stocktwit_sentiment_count_1',
       'stocktwit_sentiment_mean', 'Null_value_stocktwit'],
      dtype='object')

In [31]:
data.shape

(3799, 20)

In [32]:
def train_val_test_split(data):
    size=int(len(data)*0.6)
    # for train data will be collected from each country's data which index is from 0-size (60%)
    x_train = data.drop(columns=['direction']).iloc[0:size] 
    # for validation data 
    rest = data.iloc[size:]

    size2 = int(len(rest)*0.3)
    x_val = rest.drop(columns=['direction']).iloc[0:size2]
    # for test data
    x_test = rest.drop(columns=['direction']).iloc[size2:] 

    y_train = data['direction'].iloc[0:size] 
    y_val = rest['direction'].iloc[0:size2]
    y_test =  rest['direction'].iloc[size2:]

    return x_train, x_val, x_test, y_train, y_val, y_test

In [33]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def reshape_train_data(data):
  # Prepare the data
  scaler = StandardScaler()

  # Group the data by stock_symbol
  grouped = data.groupby('stock_symbol')

  X_list = []
  y_list = []
  Z_list = []

  seq_length = 1

  for stock_symbol, group in grouped:
      # Scale the data
      scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction'], axis=1))
      scaled_y = np.array(group['direction'])
      scaled_z = np.array(group['Close'])


      X = []
      y = []
      Z = []

      for i in range(seq_length, len(scaled_x)):
          X.append(scaled_x[i-seq_length:i])
          y.append(scaled_y[i])
          Z.append(scaled_z[i])

      X_list.append(np.array(X))
      y_list.append(np.array(y))
      Z_list.append(np.array(Z))

  # Concatenate the data for all stocks
  X_train = np.concatenate(X_list, axis=0)
  y_train = np.concatenate(y_list, axis=0)
  Close_train = np.concatenate(Z_list, axis=0)

  return X_train, y_train, Close_train




In [34]:


def reshape_val_data(data):
  # Prepare the data
  scaler = StandardScaler()

  # Group the data by stock_symbol
  grouped = data.groupby('stock_symbol')

  X_list = []
  y_list = []
  Z_list = []

  seq_length = 1

  for stock_symbol, group in grouped:
      # Scale the data
      scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction'], axis=1))
      scaled_y = np.array(group['direction'])
      scaled_z = np.array(group['Close'])

      X = []
      y = []
      Z = []

      for i in range(seq_length, len(scaled_x)):
          X.append(scaled_x[i-seq_length:i])
          y.append(scaled_y[i])
          Z.append(scaled_z[i])

      X_list.append(np.array(X))
      y_list.append(np.array(y))
      Z_list.append(np.array(Z))

  # Concatenate the data for all stocks
  X_val = np.concatenate(X_list, axis=0)
  y_val = np.concatenate(y_list, axis=0)
  Close_val = np.concatenate(Z_list, axis=0)

  return X_val, y_val, Close_val

  


In [35]:


def reshape_test_data(data):
  # Prepare the data
  scaler = StandardScaler()

  # Group the data by stock_symbol
  grouped = data.groupby('stock_symbol')

  X_list = []
  y_list = []
  Z_list = []

  seq_length = 1

  for stock_symbol, group in grouped:
      # Scale the data
      scaled_x = scaler.fit_transform(group.drop(['stock_symbol','Date','direction'], axis=1))
      scaled_y = np.array(group['direction'])
      scaled_z = np.array(group['Close'])

      X = []
      y = []
      Z = []

      for i in range(seq_length, len(scaled_x)):
          X.append(scaled_x[i-seq_length:i])
          y.append(scaled_y[i])
          Z.append(scaled_z[i])

      X_list.append(np.array(X))
      y_list.append(np.array(y))
      Z_list.append(np.array(Z))

  # Concatenate the data for all stocks
  X_test = np.concatenate(X_list, axis=0)
  y_test = np.concatenate(y_list, axis=0)
  Close_test = np.concatenate(Z_list, axis=0)

  return X_test, y_test, Close_test

  


# Loop

In [36]:
stock=list(set(data.stock_symbol))
stock.sort()

In [37]:
#stock

In [38]:
results_dict = {}

In [39]:
labels = []

In [40]:
test_data_frame = []

In [41]:
Close_test_frame = []

In [42]:

# loop each country_Region and split the data into train and test data 

for x in stock:
    stock_data = data[data['stock_symbol']==x]
    # applied the function I created above
    x_train, x_val, x_test, y_train, y_val, y_test=train_val_test_split(stock_data) 

    #combine X train and Y train as train data 
    train_data = pd.concat([x_train, y_train], axis = 1)
    val_data = pd.concat([x_val, y_val], axis = 1)
    test_data = pd.concat([x_test, y_test], axis = 1)

    test_data_frame.append(test_data)

    X_train, y_train, Close_train = reshape_train_data(train_data)
    X_val, y_val, Close_val = reshape_val_data(val_data)
    X_test, y_test, Close_test = reshape_test_data(test_data)

    Close_test_frame.append(Close_test)

    #build
    model = keras.Sequential()
    model.add(layers.LSTM(100, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(layers.LSTM(100, return_sequences=False))
    #model.add(layers.LSTM(256, return_sequences=False))
    #model.add(layers.LSTM(256, return_sequences=False))
    #model.add(layers.Dense(50))
    model.add(layers.Dense(25))
    model.add(layers.Dense(1,activation='sigmoid'))

    # Compile the model
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['accuracy'])
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), #validation_split=0.2, 
                    verbose=1, epochs=10, shuffle=False)

    scores = model.evaluate(X_test, y_test, verbose=2)
    print(x,"-Accuracy: %.2f%%" % (scores[1]*100))

    # Get prediction
    predicted_probabilities = model.predict(X_test)
    predicted_labels = [1 if prob >= 0.5 else 0 for prob in predicted_probabilities]
    labels.append(predicted_labels)

    Acc = accuracy_score(y_test, predicted_labels)
    class_report = classification_report(y_test, predicted_labels)
    class_report_lines = class_report.split('\n')
    precision_0 = class_report_lines[2].split()[1]
    precision_1 = class_report_lines[3].split()[1]

    print(Acc, precision_0, precision_1)

    # Add the results to the dictionary
    results_dict[x] = {'accuracy': Acc, 'precision_0': precision_0, 'precision_1': precision_1}

Epoch 1/10
3/3 [==============================] - 6s 534ms/step - loss: 0.6951 - accuracy: 0.4156 - val_loss: 0.6946 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 30ms/step - loss: 0.6891 - accuracy: 0.6104 - val_loss: 0.6928 - val_accuracy: 0.4286
Epoch 3/10
3/3 [==============================] - 0s 22ms/step - loss: 0.6846 - accuracy: 0.6494 - val_loss: 0.6907 - val_accuracy: 0.5000
Epoch 4/10
3/3 [==============================] - 0s 21ms/step - loss: 0.6801 - accuracy: 0.6234 - val_loss: 0.6883 - val_accuracy: 0.5000
Epoch 5/10
3/3 [==============================] - 0s 23ms/step - loss: 0.6753 - accuracy: 0.6104 - val_loss: 0.6857 - val_accuracy: 0.5000
Epoch 6/10
3/3 [==============================] - 0s 21ms/step - loss: 0.6701 - accuracy: 0.6104 - val_loss: 0.6827 - val_accuracy: 0.5000
Epoch 7/10
3/3 [==============================] - 0s 22ms/step - loss: 0.6644 - accuracy: 0.6104 - val_loss: 0.6793 - val_accuracy: 0.5000
Epoch 8/10
3/3 [==========

2/2 [==============================] - 2s 9ms/step
0.5675675675675675 0.54 0.58
Epoch 1/10
3/3 [==============================] - 5s 503ms/step - loss: 0.6931 - accuracy: 0.4675 - val_loss: 0.6879 - val_accuracy: 0.7143
Epoch 2/10
3/3 [==============================] - 0s 23ms/step - loss: 0.6856 - accuracy: 0.5584 - val_loss: 0.6820 - val_accuracy: 0.6429
Epoch 3/10
3/3 [==============================] - 0s 23ms/step - loss: 0.6789 - accuracy: 0.5844 - val_loss: 0.6756 - val_accuracy: 0.6429
Epoch 4/10
3/3 [==============================] - 0s 34ms/step - loss: 0.6719 - accuracy: 0.5974 - val_loss: 0.6684 - val_accuracy: 0.7143
Epoch 5/10
3/3 [==============================] - 0s 32ms/step - loss: 0.6645 - accuracy: 0.6104 - val_loss: 0.6601 - val_accuracy: 0.7143
Epoch 6/10
3/3 [==============================] - 0s 24ms/step - loss: 0.6566 - accuracy: 0.6234 - val_loss: 0.6508 - val_accuracy: 0.7143
Epoch 7/10
3/3 [==============================] - 0s 31ms/step - loss: 0.6486 - accur

2/2 [==============================] - 1s 8ms/step
0.5135135135135135 0.54 0.50
Epoch 1/10
3/3 [==============================] - 5s 596ms/step - loss: 0.6918 - accuracy: 0.5844 - val_loss: 0.6929 - val_accuracy: 0.5000
Epoch 2/10
3/3 [==============================] - 0s 46ms/step - loss: 0.6853 - accuracy: 0.5974 - val_loss: 0.6919 - val_accuracy: 0.5000
Epoch 3/10
3/3 [==============================] - 0s 32ms/step - loss: 0.6791 - accuracy: 0.6494 - val_loss: 0.6909 - val_accuracy: 0.5714
Epoch 4/10
3/3 [==============================] - 0s 31ms/step - loss: 0.6720 - accuracy: 0.6364 - val_loss: 0.6900 - val_accuracy: 0.5714
Epoch 5/10
3/3 [==============================] - 0s 45ms/step - loss: 0.6635 - accuracy: 0.6364 - val_loss: 0.6894 - val_accuracy: 0.5714
Epoch 6/10
3/3 [==============================] - 0s 39ms/step - loss: 0.6535 - accuracy: 0.6364 - val_loss: 0.6897 - val_accuracy: 0.5714
Epoch 7/10
3/3 [==============================] - 0s 39ms/step - loss: 0.6423 - accur

In [43]:
results_dict

{'AAPL': {'accuracy': 0.5405405405405406,
  'precision_0': '1.00',
  'precision_1': '0.47'},
 'AMGN': {'accuracy': 0.6486486486486487,
  'precision_0': '0.77',
  'precision_1': '0.47'},
 'AXP': {'accuracy': 0.40540540540540543,
  'precision_0': '0.50',
  'precision_1': '0.35'},
 'BA': {'accuracy': 0.4864864864864865,
  'precision_0': '0.25',
  'precision_1': '0.52'},
 'CAT': {'accuracy': 0.5675675675675675,
  'precision_0': '0.54',
  'precision_1': '0.58'},
 'CRM': {'accuracy': 0.5135135135135135,
  'precision_0': '0.54',
  'precision_1': '0.50'},
 'CSCO': {'accuracy': 0.4864864864864865,
  'precision_0': '0.62',
  'precision_1': '0.31'},
 'CVX': {'accuracy': 0.5405405405405406,
  'precision_0': '0.45',
  'precision_1': '0.65'},
 'DIS': {'accuracy': 0.5405405405405406,
  'precision_0': '0.64',
  'precision_1': '0.40'},
 'DOW': {'accuracy': 0.5405405405405406,
  'precision_0': '0.67',
  'precision_1': '0.50'},
 'GS': {'accuracy': 0.5405405405405406,
  'precision_0': '0.62',
  'precision

In [44]:
df_1 = pd.DataFrame.from_dict(results_dict)

In [45]:
df_1

,AAPL,AMGN,AXP,BA,CAT,CRM,CSCO,CVX,DIS,DOW,...,MMM,MSFT,NKE,PG,TRV,UNH,V,VZ,WBA,WMT
accuracy,0.540541,0.648649,0.405405,0.486486,0.567568,0.513514,0.486486,0.540541,0.540541,0.540541,...,0.648649,0.540541,0.459459,0.486486,0.540541,0.594595,0.594595,0.513514,0.540541,0.594595
precision_0,1.00,0.77,0.50,0.25,0.54,0.54,0.62,0.45,0.64,0.67,...,0.71,0.58,0.39,0.50,0.55,0.63,0.58,0.50,0.62,0.80
precision_1,0.47,0.47,0.35,0.52,0.58,0.50,0.31,0.65,0.40,0.50,...,0.60,0.52,0.53,0.48,0.53,0.56,0.61,0.55,0.44,0.52


In [46]:
predict_labels = [x for y in labels for x in y]

In [47]:
len(predict_labels)

1073

In [48]:
test_data = pd.concat(test_data_frame)

In [49]:
close_test = [x for y in Close_test_frame for x in y]
len(close_test)

1073

In [50]:
#test_data

In [51]:
# Group the DataFrame by stock_symbol
test_group = test_data.groupby("stock_symbol")

# Remove the first 7 days of data for each stock symbol
test1 = test_group.apply(lambda x: x.iloc[1:])
test1.reset_index(drop=True, inplace=True)
#test1

In [52]:
test1['Prediction'] = predict_labels
test1['Close_test'] = close_test

In [53]:
#test1

In [54]:
# Select the rows where prediction is equal to 1
prediction_1 = test1[test1["Prediction"] == 1].copy()

# Compute the subtraction and sum operation on the selected rows

return_rate = ((100/prediction_1["Open"])*prediction_1["Close"]*(1-0.001)-100*(1+0.001))/100*(1+0.001)
profit = (100/prediction_1["Open"])*prediction_1["Close"]*(1-0.001)-100*(1+0.001)
#result = prediction_1["Open_t_1_test"].sub(prediction_1["Open"]).sum()


# overall_return_rate = 1
# for rate in result:
#     overall_return_rate *= (1 + rate)

# # Step 3: Convert the overall return rate back to a percentage
# overall_return_percentage = (overall_return_rate - 1) * 100

# print("Overall return rate:", overall_return_percentage, "%")


# Print the result
print(return_rate, profit)

print(sum(return_rate), sum(profit))
print(return_rate.mean(), profit.mean())

0       0.014171
2      -0.008578
3      -0.007680
4       0.057262
5      -0.013962
          ...   
1066   -0.003513
1067    0.005905
1069   -0.010913
1070   -0.003599
1072   -0.001932
Length: 627, dtype: float64 0       1.415643
2      -0.856954
3      -0.767253
4       5.720438
5      -1.394766
          ...   
1066   -0.350969
1067    0.589865
1069   -1.090183
1070   -0.359515
1072   -0.192976
Length: 627, dtype: float64
0.11142563715070324 11.131432282787515
0.00017771233995327423 0.01775348051481262


In [55]:
test1.loc[test1['Prediction'] == 1, 'return rate'] = return_rate
test1.loc[test1['Prediction'] == 1, 'profit'] = profit

In [56]:
#test1

In [57]:
def calculate_profit(group):
  return_rate  = ((100/group["Open"])*group["Close"]*(1-0.001)-100*(1+0.001))/100*(1+0.001)
  profit = (100/group["Open"])*group["Close"]*(1-0.001)-100*(1+0.001)
  x = return_rate.sum()
  y = return_rate.mean()
  m = profit.sum()
  n = profit.mean()
  return x,y,m,n

# group the DataFrame by stock symbol and calculate the accuracy for each group
profit_df = prediction_1[['stock_symbol','Prediction','Open','Close']].groupby('stock_symbol').apply(calculate_profit).reset_index(name='profit')

# Extract the x and y values from the 'profit' column
profit_df[['return rate sum', 'return rate mean','profit sum','profit mean']] = profit_df['profit'].apply(pd.Series)

# Drop the 'profit' column
profit_df = profit_df.drop('profit', axis=1)

# print the resulting DataFrame
#profit_df

In [58]:
profit_df['profit sum'].sum()

11.131432282787515

In [59]:
def calculate_accuracy(group):
    return (group['Prediction'] == group['direction']).mean()

# group the DataFrame by stock symbol and calculate the accuracy for each group
accuracy_df = test1[['stock_symbol','direction','Prediction']].groupby('stock_symbol').apply(calculate_accuracy).reset_index(name='Accuracy')

# print the resulting DataFrame
accuracy_df

,stock_symbol,Accuracy
0,AAPL,0.540541
1,AMGN,0.648649
2,AXP,0.405405
3,BA,0.486486
4,CAT,0.567568
5,CRM,0.513514
6,CSCO,0.486486
7,CVX,0.540541
8,DIS,0.540541
9,DOW,0.540541


In [60]:
accuracy_df.Accuracy.mean()

0.5265610438024231

In [61]:
# define a function to calculate the precision for a group of rows
def calculate_precision(group, label):
    tp = ((group['Prediction'] == label) & (group['direction'] == label)).sum()
    fp = ((group['Prediction'] == label) & (group['direction'] != label)).sum()
    if (tp + fp) == 0:
        return 0
    else:
        return tp / (tp + fp)

# group the DataFrame by stock symbol and calculate the precision for each group
precision_0 = test1[['stock_symbol','direction','Prediction']].groupby('stock_symbol').apply(lambda x: calculate_precision(x, 0)).reset_index(name='Precision_0')
precision_1 = test1[['stock_symbol','direction','Prediction']].groupby('stock_symbol').apply(lambda x: calculate_precision(x, 1)).reset_index(name='Precision_1')

# merge the precision DataFrames with the accuracy DataFrame
result_df = pd.merge(accuracy_df, precision_0, on='stock_symbol').merge(precision_1, on='stock_symbol')

# print the resulting DataFrame
result_df

,stock_symbol,Accuracy,Precision_0,Precision_1
0,AAPL,0.540541,1.000000,0.468750
1,AMGN,0.648649,0.772727,0.466667
2,AXP,0.405405,0.500000,0.347826
3,BA,0.486486,0.250000,0.515152
4,CAT,0.567568,0.538462,0.583333
5,CRM,0.513514,0.538462,0.500000
6,CSCO,0.486486,0.619048,0.312500
7,CVX,0.540541,0.450000,0.647059
8,DIS,0.540541,0.636364,0.400000
9,DOW,0.540541,0.666667,0.500000


In [62]:
result_df.Precision_0.mean()

0.5553884568740725

In [63]:
result_df.Precision_1.mean()

0.5075564566239175

In [64]:
result_df.Accuracy.mean()

0.5265610438024231

In [65]:
final_result = result_df.merge(profit_df, on=['stock_symbol'], how='left')

In [66]:
final_result

,stock_symbol,Accuracy,Precision_0,Precision_1,return rate sum,return rate mean,profit sum,profit mean
0,AAPL,0.540541,1.000000,0.468750,-0.065620,-0.002051,-6.555434,-0.204857
1,AMGN,0.648649,0.772727,0.466667,0.038422,0.002561,3.838404,0.255894
2,AXP,0.405405,0.500000,0.347826,-0.029344,-0.001276,-2.931437,-0.127454
3,BA,0.486486,0.250000,0.515152,0.054479,0.001651,5.442413,0.164922
4,CAT,0.567568,0.538462,0.583333,0.068124,0.002838,6.805570,0.283565
5,CRM,0.513514,0.538462,0.500000,0.142151,0.005923,14.200869,0.591703
6,CSCO,0.486486,0.619048,0.312500,-0.025869,-0.001617,-2.584326,-0.161520
7,CVX,0.540541,0.450000,0.647059,0.097059,0.005709,9.696182,0.570364
8,DIS,0.540541,0.636364,0.400000,-0.039935,-0.002662,-3.989517,-0.265968
9,DOW,0.540541,0.666667,0.500000,-0.066746,-0.002384,-6.667896,-0.238139


In [67]:
test1.to_csv('drive/My Drive/thesis/dataset/final_prediction_dataset/ [New] final_prediction_separate_model [open to close] - price+macd+twitter+stocktwit.csv')

In [68]:
final_result.to_csv('drive/My Drive/thesis/dataset/final_prediction/ [New] final_prediction_separate_models [open to close] - price+macd+twitter+stocktwit.csv')